In [2]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup as bs
import csv 
import requests
import pandas as pd
import re
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.common.exceptions import UnexpectedAlertPresentException
import urllib.request
import random

In [3]:
#수집하고 싶은 기간 
#2019년 12월 31일까진 뉴스기사 아주 적음 
first_days = pd.date_range('2005/01/01', '2020/12/31', freq='2AS')
last_days = pd.date_range('2005/01/01', '2020/12/31', freq='2A')
first_days, last_days

(DatetimeIndex(['2005-01-01', '2007-01-01', '2009-01-01', '2011-01-01',
                '2013-01-01', '2015-01-01', '2017-01-01', '2019-01-01'],
               dtype='datetime64[ns]', freq='2AS-JAN'),
 DatetimeIndex(['2005-12-31', '2007-12-31', '2009-12-31', '2011-12-31',
                '2013-12-31', '2015-12-31', '2017-12-31', '2019-12-31'],
               dtype='datetime64[ns]', freq='2A-DEC'))

In [4]:
#첫번째 input으로 검색 키워드를 설정합니다 (e.g., "자동차")
keyword = '메타버스'

#url을 설정합니다
url = 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query={}'.format(keyword)

# 크롬 드라이버 선언
driver = webdriver.Chrome("chromedriver.exe") #"chromedriver.exe" 대신에 현재 컴퓨터의 크롬드라이버 위치가 들어가야 합니다
driver.implicitly_wait(1)

# 네이버 VIEW 블로그 검색창 이동
driver.get(url)
time.sleep(1)

<ipython-input-4-fe9877e533c2>:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chromedriver.exe") #"chromedriver.exe" 대신에 현재 컴퓨터의 크롬드라이버 위치가 들어가야 합니다


In [5]:
title = []
url = []
journal = []
time1 = []
time2 = []

In [6]:
# 검색 옵션 클릭하기
driver.find_element_by_xpath('//*[@id="snb"]/div[1]/div/div[2]/a').click()
time.sleep(0.8)
for k in range(len(first_days)):
    
    try :
        # 직접입력 클릭하기
        driver.find_element_by_xpath('//*[@id="snb"]/div[2]/ul/li[2]/div/div[1]/a[9]').click()
        time.sleep(2)    

        #시작 날짜 지정
        #년(year) 클릭 
        driver.find_element_by_xpath('//*[@id="snb"]/div[2]/ul/li[2]/div/div[3]/div[2]/div[1]/div/div/div/ul/li[{0}]/a'.format(first_days.year[k]-1988)).click()
        driver.find_element_by_xpath('//*[@id="snb"]/div[2]/ul/li[2]/div/div[3]/div[2]/div[1]/div/div/div/ul/li[{0}]/a'.format(first_days.year[k]-1989)).click()
        time.sleep(1)
        #월(momth) 클릭
        driver.find_element_by_xpath('//*[@id="snb"]/div[2]/ul/li[2]/div/div[3]/div[2]/div[2]/div/div/div/ul/li[{0}]'.format(first_days.month[k])).click()
        #일 (day) 클릭
        driver.find_element_by_xpath('//*[@id="snb"]/div[2]/ul/li[2]/div/div[3]/div[2]/div[3]/div/div/div/ul/li[{0}]'.format(first_days.day[k])).click()
        #끝 날짜 클릭
        driver.find_element_by_xpath('//*[@id="snb"]/div[2]/ul/li[2]/div/div[3]/div[1]/span[3]/a').click()
        #년(year) 클릭 
        driver.find_element_by_xpath('//*[@id="snb"]/div[2]/ul/li[2]/div/div[3]/div[2]/div[1]/div/div/div/ul/li[{0}]'.format(last_days.year[k]-1989)).click()
        time.sleep(0.3)
        #월(momth) 클릭
        driver.find_element_by_xpath('//*[@id="snb"]/div[2]/ul/li[2]/div/div[3]/div[2]/div[2]/div/div/div/ul/li[{0}]'.format(last_days.month[k])).click()
        #일 (day) 클릭
        driver.find_element_by_xpath('//*[@id="snb"]/div[2]/ul/li[2]/div/div[3]/div[2]/div[3]/div/div/div/ul/li[{0}]/a'.format(last_days.day[k])).click()
        #적용 클릭
        driver.find_element_by_xpath('//*[@id="snb"]/div[2]/ul/li[2]/div/div[3]/div[3]/button').click()
        #오래된순 클릭하기
        driver.find_element_by_xpath('//*[@id="snb"]/div[2]/ul/li[1]/div/div/a[3]').click()
    except : 
        pass
    
    prev_next = driver.find_element_by_class_name('sc_page_inner') #아래 a가 들어있는 클래스
    buttons = prev_next.find_elements_by_tag_name('a') # 페이지 넘어가는 숫자 1~n   

    page_num = 400 #수집하고 싶은 페이지 개수 지정 (게시물 개수 아님)

    if len(buttons) == 0: # 지정한 조건으로 게시물이 하나도 없을. 인식하는데 오래 걸림.
        pass

    else: #게시물이 있으면 url 수집함 
        try: # 중간중간 에러가 발생해서 전체를 try except안에 넣음
            for p in range(1,page_num+1): #페이지마다 반복해서 수집 #1부터 page_num까지

                # url 수집
                soup = bs(driver.page_source, 'lxml') 
                
                table = soup.find('ul',{'class' : 'list_news'})
                li_list = table.find_all('li', {'id': re.compile('sp_nws.*')})
                area_list = [li.find('div', {'class' : 'news_area'}) for li in li_list]
                a_list = [area.find('a', {'class' : 'news_tit'}) for area in area_list]
                journal_list = [area.find('div', {'class' : 'info_group'}) for area in area_list]

                for n in a_list:
                    try :
                        title.append(n.get('title'))
                    except :
                        title.append('')
                    try :
                        url.append(n.get('href'))
                    except :
                        url.append('')

                for n in journal_list:
                    try :
                        journal.append(n.find('a', class_='info press').text)
                    except :
                        journal.append('')
                    try :
                        time1.append(n.find_all('span', class_='info'))
                    except :
                        time1.append('')
                    try :
                        time2.append(n.find('span', class_='info').text)
                    except :
                        time2.append('')

                prev_next = driver.find_element_by_class_name('sc_page_inner') #아래 a가 들어있는 클래스
                buttons = prev_next.find_elements_by_tag_name('a') # 페이지 넘어가는 숫자 1~n   


                # 페이지 이동 
                # 페이지는 유형이 총 3가지가 있음
                # 이전X 다음O 첫 페이지 / 이전O 다음O 중간 페이지 / 이전O 다음X 마지막 페이지
                # 유형에 따라 buttons수가 달라짐

                if p<=10 & len(buttons)<12: #첫 페이지. #buttons의 수는 페이지수+다음버튼임 따라서 첫페이지라면 buttons수는 11(혹은 11미만)
                    if p%10 != 0: #10의 배수가 아니라면 
                        buttons[p].click() #p는 for문에서 1부터 시작하여 돌고있음. 여기서 button[0]은 1번째 페이지, button[1]은 2번째 페이지
                    else: #10의 배수라면 다음 버튼을 눌러줌
                        prev_next.find_element_by_css_selector('a.pgR').click()

                elif len(buttons) == 12: #이후 페이지 #다음 페이지라면 이전버튼+페이지수+다음버튼임 따라서 buttons수는 12
                    if p%10 != 0:
                        buttons[p%10+1].click() # p가 11이라면 button[2]를 누르게되는 것. 여기서 button[0]은 이전버튼, buttton[1]은 11페이지, button[2]는 12페이지 
                    else: 
                        prev_next.find_element_by_css_selector('a.pgR').click()

                elif p>10 & len(buttons)<12 : #마지막 페이지 #이전버튼+페이지수임 따라서 buttons수는 11이하
                    buttons[p%10+1].click() 
        except: 
            pass
  


    print('{0}년 {1}월 총'.format(first_days.year[k], first_days.month[k]), len(title),' 개 뉴스 url 수집')
#     driver.find_element_by_xpath('//*[@id="snb"]/div[2]/ul/li[2]/div/div[1]/a[9]').click()
    time.sleep(0.8)    



<ipython-input-6-3124af2bf7b1>:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="snb"]/div[1]/div/div[2]/a').click()
<ipython-input-6-3124af2bf7b1>:8: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="snb"]/div[2]/ul/li[2]/div/div[1]/a[9]').click()
<ipython-input-6-3124af2bf7b1>:13: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="snb"]/div[2]/ul/li[2]/div/div[3]/div[2]/div[1]/div/div/div/ul/li[{0}]/a'.format(first_days.year[k]-1988)).click()
<ipython-input-6-3124af2bf7b1>:14: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="snb"]/div[2]/ul/li[2]/div/div[3]/div[2]/div[1]/div/div/div/ul/li[{0}]/a'.format(first_days.year[k]-1989)).click(

2005년 1월 총 1  개 뉴스 url 수집
2007년 1월 총 20  개 뉴스 url 수집
2009년 1월 총 41  개 뉴스 url 수집
2011년 1월 총 41  개 뉴스 url 수집


KeyboardInterrupt: 

In [7]:
len(set(url)), len(url)

(41, 41)

In [8]:
import pandas as pd
df = pd.DataFrame()
df['title'] = title
df['url'] = url
df['journal'] = journal
df['time'] = time1
df['time2'] = time2
df

,title,url,journal,time,time2
0,[대한민국SW대상]주요 전시관-소프트비전관,https://news.naver.com/main/read.naver?mode=LS...,전자신문,[[2005.12.01.]],2005.12.01.
1,안시 청이 말하는 가상공간 비즈니스 성공의 비밀 (2),http://www.zdnet.co.kr/news/internet/entertain...,ZDNet Korea,[[2007.01.03.]],2007.01.03.
2,"세컨드 라이프, 최고의 ""사업 계획"" 선정",http://www.zdnet.co.kr/news/internet/entertain...,ZDNet Korea,[[2007.02.21.]],2007.02.21.
3,[글로벌 IT이슈 진단]버츄얼 환경 급속 확산,http://www.etnews.co.kr/news/detail.html?id=20...,전자신문,[[2007.04.25.]],2007.04.25.
4,방송통신융합은 디지털콘텐츠 활성화를 위한 기회,https://news.naver.com/main/read.naver?mode=LS...,,"[[[<img alt="""" class=""thumb bg_default_press"" ...",정보통신부
5,미래 콘텐츠 핵심은 `가상`,http://www.dt.co.kr/contents.html?article_no=2...,디지털타임스,[[2007.05.11.]],2007.05.11.
6,[IT 재팬 Report] 제2의 도쿄 e곳에 우뚝,http://www.dt.co.kr/contents.html?article_no=2...,디지털타임스,[[2007.06.12.]],2007.06.12.
7,[권택민의 디씨 톡톡] 환상세계와 온라인 게임,http://www.cbs.co.kr/Nocut/Show.asp?IDX=558581,노컷뉴스,[[2007.07.09.]],2007.07.09.
8,공상과학 효과가 첨단기술에 미치는 영향,"http://www.zdnet.co.kr/news/digital/0,39030978...",ZDNet Korea,[[2007.07.09.]],2007.07.09.
9,"[벤처정책 10년, 벤처산업 10년]소프트웨어-우대칼스",http://www.etnews.co.kr/news/detail.html?id=20...,전자신문,[[2007.07.26.]],2007.07.26.


In [9]:
df.to_csv("url1.csv", encoding = 'utf-8-sig')